In [1]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import nltk     
import re
import string
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import graphviz 
from sklearn import tree
from sklearn import metrics
from wordcloud import WordCloud
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
DATA_DIR = os.path.join( '..', 'data')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'processed')

In [2]:
file_path=os.path.join(PROCESSED_DATA_DIR,"sms.csv")
sms=pd.read_csv(file_path)
sms.head(2)




,Label,SMS,lenght,Tokens,lemmatized_text,SMS_without_stopwords,processed_text
0,0,"gone until jurong point, crazy.. available onl...",111,"['gone', 'until', 'jurong', 'point', ',', 'cra...","['go', 'until', 'jurong', 'point', ',', 'crazy...","gone jurong point, crazy.. available bugis gre...","gone until jurong point, crazy.. available onl..."
1,0,ok lar... joking wif you oni...,29,"['ok', 'lar', '...', 'joking', 'wif', 'you', '...","['ok', 'lar', '...', 'joking', 'wif', 'you', '...",ok lar... joking wif oni...,ok lar... joking wif you oni...


In [3]:
#Para entrenar un modelo de regresión logística, necesitas convertir tus datos de texto en
#características numéricas. Puedes hacer esto utilizando un enfoque de "bag of words"
#(bolsa de palabras) con CountVectorizer:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sms['SMS'])
tamanio_X = X.size
X_new=['gone','want','get','one','please','message','send','call','come','know','free','mobile','claim','reply','good','stop','new','got','like','text']
list_key=list(range(8))
new_columns = list_key[0:8]
X_new=X[new_columns]

In [4]:
# Supongamos que ya tienes 'sms1' y 'list_key' definidos
# También, supongo que 'sms1' es un DataFrame de Pandas.

# Crear el vectorizador y ajustar/transformar los datos
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sms['SMS'])

# Obtener las columnas correspondientes a las palabras en 'list_key'
#list_key = ["want","get"]
list_key = X_new=['gone','want','get','one','please','message','send','call','come','know','free','mobile','claim','reply','good','stop','new','got','like','text']
selected_columns = [vectorizer.vocabulary_[word] for word in list_key if word in vectorizer.vocabulary_]

# Seleccionar solo las columnas relevantes
X_new = X[:, selected_columns]

# Convertir X_new a un DataFrame de Pandas
X_new_df = pd.DataFrame(X_new.toarray(), columns=['X_' + str(i) for i in selected_columns])

# Concatenar X_new_df con el conjunto de datos original sms1
sms = pd.concat([sms, X_new_df], axis=1)

# Verificar las dimensiones de X_new
print(X_new.shape)

# Verificar las dimensiones de sms1 después de la concatenación
print(sms.shape)
sms.head(2)

(5263, 20)
(5263, 27)


,Label,SMS,lenght,Tokens,lemmatized_text,SMS_without_stopwords,processed_text,X_3615,X_8366,X_3544,...,X_3390,X_5148,X_2081,X_6480,X_3619,X_7365,X_5391,X_3637,X_4670,X_7703
0,0,"gone until jurong point, crazy.. available onl...",111,"['gone', 'until', 'jurong', 'point', ',', 'cra...","['go', 'until', 'jurong', 'point', ',', 'crazy...","gone jurong point, crazy.. available bugis gre...","gone until jurong point, crazy.. available onl...",1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,ok lar... joking wif you oni...,29,"['ok', 'lar', '...', 'joking', 'wif', 'you', '...","['ok', 'lar', '...', 'joking', 'wif', 'you', '...",ok lar... joking wif oni...,ok lar... joking wif you oni...,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#check all the columns
sms.columns


Index(['Label', 'SMS', 'lenght', 'Tokens', 'lemmatized_text',
       'SMS_without_stopwords', 'processed_text', 'X_3615', 'X_8366', 'X_3544',
       'X_5601', 'X_5957', 'X_5049', 'X_6814', 'X_1841', 'X_2177', 'X_4498',
       'X_3390', 'X_5148', 'X_2081', 'X_6480', 'X_3619', 'X_7365', 'X_5391',
       'X_3637', 'X_4670', 'X_7703'],
      dtype='object')

In [6]:
sms_final =sms[['Label','X_3615', 'X_8366', 'X_3544',
       'X_5601', 'X_5957', 'X_5049', 'X_6814', 'X_1841', 'X_2177', 'X_4498',
       'X_3615', 'X_8366', 'X_3544', 'X_5601', 'X_5957', 'X_5049', 'X_6814',
       'X_1841', 'X_2177', 'X_4498', 'X_3390', 'X_5148', 'X_2081', 'X_6480',
       'X_3619', 'X_7365', 'X_5391', 'X_3637', 'X_4670', 'X_7703']]

In [7]:
#sms_final separated in train validation and test
sms_train, sms_test = train_test_split(sms_final, test_size=0.2, random_state=42)

sms_train, sms_val = train_test_split(sms_train, test_size=0.2, random_state=42)


In [8]:
# # Inicializar y entrenar un vectorizador TF-IDF
# vectorizer = TfidfVectorizer(stop_words='english')
# sms_train = vectorizer.fit_transform(sms_train['SMS']).todense()
# sms_val=vectorizer.transform(sms_val['SMS']).todense()
# sms_test = vectorizer.transform(sms_test['SMS']).todense()

In [9]:
#save in csv the train, validation and test in the folder processed
sms_train.to_csv(os.path.join(PROCESSED_DATA_DIR,"sms_train.csv"),index=False)
sms_val.to_csv(os.path.join(PROCESSED_DATA_DIR,"sms_val.csv"),index=False)
sms_test.to_csv(os.path.join(PROCESSED_DATA_DIR,"sms_test.csv"),index=False)
